In [4]:
from langchain.llms import OpenAI
import os
from langchain_community.document_loaders import WikipediaLoader
from langchain_openai import ChatOpenAI
from langchain.prompts import (
                                PromptTemplate,
                                ChatPromptTemplate, 
                                SystemMessagePromptTemplate, 
                                HumanMessagePromptTemplate, 
                                AIMessagePromptTemplate )
import requests

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


def extract_keywords(text, n = 2):
    response = llm.invoke("What are the top {n} most important keywords to search for on Wikipedia to verify claim" + text + " Please give the answer in a common delimited sentence with no space from the most important to the least important.")
    print(response)
    response = dict(response)['content'].split(',')
    return response

def get_wikipedia_info(keywords, max_chars = 4000):
    result = {}
    for keyword in keywords:
          docs = WikipediaLoader(query=keyword, load_max_docs=1, doc_content_chars_max=max_chars).load()
          if docs:
            wiki_key = docs[0].metadata['title']
            result[wiki_key] = docs[0]
    return result

def get_wikipedia_info_from_text(text, n_keywords =2, max_chars = 4000):
     keywords = extract_keywords(text)
     wiki_info = get_wikipedia_info(keywords)
     return wiki_info


In [6]:
print(get_wikipedia_info(['Paris']))

{'Paris': Document(page_content="Paris is the capital and most populous city of France. With an official estimated population of 2,102,650 residents as of 1 January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-most populated city in the European Union and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. For its leading role in the arts and sciences, as well as its early and extensive system of street lighting, in the 19th century, it became known as the City of Light. \nThe City of Paris is the centre of the Île-de-France region, or Paris Region, with an official estimated population of 12,271,794 inhabitants on 1 January 2023, or about 19% of the population of France. The Paris Region had a GDP of €765 billion (US$1.064 trillion, PPP) in 2021, the highest in the European Union. According to the Economist Intelligen

In [108]:
# from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# system_prompt = SystemMessage(content="You are a fact checker. Given input claim and relevant informaiton, you will respond Yes or No with a short justification citing from the wiki information. You return a json response with the keys 'response' and 'justification': {'response': '<yes_or_no>', 'justification': '<justification>'}")
# human_prompt = PromptTemplate.from_template("claim: {claim}, info: {info}")
from langchain_core.prompts import ChatPromptTemplate



chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a fact checker. Given input claim and relevant informaiton, you will respond Yes or No with a short justification citing from the wiki information. You return a json response with the keys 'response' and 'justification'"),
        ("human", "claim: {claim}, info: {info}"),
    ]
)

def compose_query(claim, wiki_info):
    def concat_wiki_info(wiki_info):
        result = ""
        for key, doc in wiki_info.items():
            result += key + ": " + doc.page_content + "\n"
        return result
    wiki_info = concat_wiki_info(wiki_info)
    return "claim: " + claim + "\n info: " + wiki_info



c = chat_template.format(claim="Beijing is the capital of China", info="")
c

"System: You are a fact checker. Given input claim and relevant informaiton, you will respond Yes or No with a short justification citing from the wiki information. You return a json response with the keys 'response' and 'justification'\nHuman: claim: Beijing is the capital of China, info: "

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a powerful fact checker that checks if the input claim is true or false given relevant information extracted from wikipeida. You should always answer 'Yes.' or 'No.', then follow by a short justification by citing from the wiki information. """       ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
from langchain.tools.render import format_tool_to_openai_function

tools = [verify_claim]
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [102]:
wiki_info = get_wikipedia_info_from_text("Beijing is the capital of china")
wiki_info

content='Beijing,Capital,China' response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 53, 'total_tokens': 60}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None} id='run-d497fc69-6096-4a39-b9b4-54dab284a435-0'


/Users/kellychen/opt/anaconda3/envs/cs6682/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/kellychen/opt/anaconda3/envs/cs6682/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


{'Beijing': Document(page_content="Beijing, alternatively romanized as Peking, is the capital of China. With about 22 million residents, Beijing is the world's most populous national capital city as well as China's second largest city after Shanghai. It is located in Northern China, and is governed as a municipality under the direct administration of the State Council with 16 urban, suburban, and rural districts. Beijing is mostly surrounded by Hebei Province with the exception of neighboring Tianjin to the southeast; together, the three divisions form the Jingjinji megalopolis and the national capital region of China.\nBeijing is a global city and one of the world's leading centres for culture, diplomacy, politics, finance, business and economics, education, research, language, tourism, media, sport, science and technology and transportation. It is home to the headquarters of most of China's largest state-owned companies and houses the largest number of Fortune Global 500 companies in

'Dune: Part Two'